In [1]:
import pandas as pd
df_1 = pd.read_csv('train.csv')
df_2 = pd.read_csv('train_add.csv')
df_test_1 = pd.read_csv('test.csv')
df_test_2 = pd.read_csv('2014_add.csv')
df_stadium = pd.read_csv('stadium.csv')
df_condition_1 = pd.read_csv('condition.csv')
df_condition_2 = pd.read_csv('condition_add.csv')         #ファイルの読み込み

In [2]:
df = pd.concat([df_1, df_2],axis=0)
df_test = pd.concat([df_test_1, df_test_2], axis=0)       #追加されてる部分を結合
df_all = pd.concat([df, df_test], axis=0)                 #ダミー変数を合わせるために、testとtrainを結合

In [3]:
df['match'].head(1)

0    第１節第１日
Name: match, dtype: object

In [4]:
stage_mapping = {'Ｊ１':1, 'Ｊ２':2}
home_dummy = pd.get_dummies(df_all['home'])
away_dummy = pd.get_dummies(df_all['away'])
df_all['stage'] = df_all['stage'].map(stage_mapping)
df_all = pd.concat([df_all, home_dummy, away_dummy], axis=1)
df_all = df_all.drop('home', axis=1)
df_all = df_all.drop('away', axis=1)               #home, away, j1 j2をダミー変数にする
df_all = df_all.drop('address', axis=1)
df_all = df_all.drop('tv', axis=1)                 #addressとtvをなくす

In [5]:
#時間も〜時開始を基準にしてダミー変数にする

time_mapping = {}
for number in range(len(df_all['time'])):
    time_mapping[df_all['time'].iloc[number]] = df_all['time'].iloc[number][0:2]  

df_all['time'] = df_all['time'].map(time_mapping)
time_dummy = pd.get_dummies(df_all['time'])
df_all = pd.concat([df_all, time_dummy], axis=1)
df_all = df_all.drop('time', axis=1)

In [6]:
#ゲームを月ごとに分けてダミー変数にする

month_mapping = {}
for number in range(len(df_all['gameday'])):
    month_mapping[df_all['gameday'].iloc[number]] = df_all['gameday'].iloc[number][0:2]

df_all['gameday'] = df_all['gameday'].map(month_mapping)
month_dummy = pd.get_dummies(df_all['gameday'])
df_all = pd.concat([df_all, month_dummy], axis=1)
df_all = df_all.drop('gameday', axis=1)

In [7]:
#節の第１日目に行われるか、第２日目に行われるか（多くの場合、土曜なのか日曜なのか）

day_mapping = {}
for number in range(len(df_all['match'])):
    if df_all['match'].iloc[number][-2:] == '１日':
        day_mapping[df_all['match'].iloc[number]] = 1
    else:
        day_mapping[df_all['match'].iloc[number]] = 2

df_all['match'] = df_all['match'].map(day_mapping)
df_all['match'].head(1)

0    1
Name: match, dtype: int64

In [8]:
df_all.head(1)

,id,match,stadium,stage,y,year,アビスパ福岡,アルビレックス新潟,カターレ富山,カマタマーレ讃岐,...,03,04,05,06,07,08,09,10,11,12
0,13994,1,ユアテックスタジアム仙台,1,18250.0,2012,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [9]:
df_new = df_all.iloc[:len(df)]
df_new_test = df_all.iloc[len(df):]               #trainとtestに再分割
df_new_test.iloc[0]

id                    15822
match                     1
stadium        ユアテックスタジアム仙台
stage                     1
y                       NaN
year                   2014
アビスパ福岡                    0
アルビレックス新潟                 0
カターレ富山                    0
カマタマーレ讃岐                  0
ガイナーレ鳥取                   0
ガンバ大阪                     0
ギラヴァンツ北九州                 0
コンサドーレ札幌                  0
サガン鳥栖                     0
サンフレッチェ広島                 0
ザスパクサツ群馬                  0
ザスパ草津                     0
ジェフユナイテッド千葉               0
ジュビロ磐田                    0
セレッソ大阪                    0
ファジアーノ岡山                  0
ベガルタ仙台                    1
モンテディオ山形                  0
ロアッソ熊本                    0
ヴァンフォーレ甲府                 0
ヴィッセル神戸                   0
京都サンガF.C.                 0
名古屋グランパス                  0
大分トリニータ                   0
                   ...     
横浜Ｆ・マリノス                  0
横浜ＦＣ                      0
水戸ホーリーホック                 0
浦和レッズ                     0
清水エスパルス             

In [10]:
df_stadium = df_stadium.rename(columns={'name':'stadium'})      #列名を合わせる
df_new = pd.merge(df_new, df_stadium, on='stadium')             #mergeの時、順番変わることに注意
df_new_test = pd.merge(df_new_test, df_stadium, on='stadium')   #スタジアム情報と試合情報をつなげる

In [11]:
df_new.drop('address', axis=1, inplace=True)
df_new.drop('stadium', axis=1, inplace=True)
df_new_test.drop('address', axis=1, inplace=True)
df_new_test.drop('stadium', axis=1, inplace=True)                #スタジアム名と住所をなくす

In [12]:
df_new_X = df_new.drop('id', axis=1)
df_new_test_X = df_new_test.drop('id', axis=1)                   #xとyにわける

In [13]:
X = df_new_X.drop('y', axis=1).values
y = df_new_X.loc[:, ['y']].values                                #ndarray型にする

In [14]:
X_test = df_new_test_X.drop('y', axis=1).values

In [17]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
from sklearn.model_selection import KFold
kf_3 = KFold(n_splits=20, shuffle=True)
from sklearn.model_selection import cross_val_score
scores_3 = cross_val_score(lr, X, y, cv=kf_3)
scores_3.mean()                                                  #線形分離した時の精度予測

0.820187496425566

In [18]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X, y)                                                     #線形分離

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [19]:
answer = []
for number in range(len(X_test)):
    answer.append([df_new_test['id'].iloc[number],
                   lr.predict(X_test[number].reshape(1, -1))[0][0]])
answer[:5]                                                       #idと予想がセットとなるリスト作成

[[15822, 13850.28579711914],
 [15840, 14343.251190185547],
 [15863, 16059.820190429688],
 [15885, 18799.603912353516],
 [15903, 15636.087829589844]]

In [20]:
df_answer = pd.DataFrame(answer)
df_answer.to_csv('j_league_6.csv', index=False)　　　　　　　　　　　#上のリストをcsvファイルにする（これで終了）

In [43]:
#正則化も試してみる

from sklearn.model_selection import train_test_split
# ホールド・アウト法による分割
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X, y, test_size=0.3)

In [44]:
# 調整したいパラメータを指定
param_grid = {'alpha': [0.3, 0.5, 0.7]}

In [45]:
# データの分割方法を指定
from sklearn.model_selection import KFold
kf_3 = KFold(n_splits=20, shuffle=True)

In [46]:
# GridSearchCVのインスタンスを生成
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge                        # Ridge回帰のクラスを読み込み
# (モデルのインスタンス, 試したいパラメータの値, 分割方法)
gs_svc = GridSearchCV(Ridge(), param_grid, cv=kf_3)

In [47]:
# GridSearchCVを利用した学習
# 最適なパラメータを自動で採用した分類器となる
gs_svc.fit(X_train_2, y_train_2)

GridSearchCV(cv=KFold(n_splits=20, random_state=None, shuffle=True),
       error_score='raise',
       estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'alpha': [0.3, 0.5, 0.7]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score='warn', scoring=None, verbose=0)

In [48]:
# test精度の平均が最も高かった組み合わせを出力
gs_svc.best_params_

{'alpha': 0.5}

In [49]:
# そのときのtest精度の平均を出力
gs_svc.best_score_

0.8216234408879675

In [50]:
# ホールド・アウト法によるtest精度を出力
gs_svc.score(X_test_2, y_test_2)

0.783675329659795

In [51]:
#微妙だったのでやめる